In [1]:
import ccxt
import pandas as pd
import polars as pl
import os

# GLOBAL VARIABLES
# Initialize the specific exchange 
EXCHANGE_NAME = "binance"
EXCHANGE = getattr(ccxt, EXCHANGE_NAME)()  # i.e. ccxt.binance()
markets = EXCHANGE.load_markets() 
TICKER_DATA_PATH = r"C:\Users\Damja\CODING_LOCAL\trading\ticker_specific_data_BINANCE"

def fetch_ohlcv_data_per_symbol(symbol, timeframe='1h', **kwargs):
    try:
        # Fetch OHLCV data for the current pairs
        ohlcv = EXCHANGE.fetch_ohlcv(symbol, timeframe=timeframe, **kwargs)
        # Convert to a DataFrame
        df = pd.DataFrame(ohlcv, columns=['Date', 'open', 'high', 'low', 'close', 'volume'])
        df['Date'] = pd.to_datetime(df['Date'], unit='ms')
        df.set_index('Date', inplace=True)
        df['usd_volume'] = df['close'] * df['volume']
        return df
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")


def update_ohlcv_data_per_symbol(symbol, timeframe='1h', timestamp=None, **kwargs):
    '''
    Update data from last 
    arguments:
        symbol: str (as given by exchange, i.e. 'BTC/USD', needs to be converted to 'BTCUSD' for data folder)
        timeframe: str
        timestamp: datetime
        kwargs: other arguments for fetch_ohlcv_data_per_symbol
    '''
    assert timestamp is not None, "Timestamp must be provided"
    data_folder_symbol = symbol.replace('/', '')
    if timeframe == '1h':
        data_folder_symbol = data_folder_symbol + '_60'
    else:
        raise Exception("Timeframe not supported")

    df = load_ohlcv_data_per_symbol(data_folder_symbol)
    df_since_timestamp = fetch_ohlcv_data_per_symbol(symbol=symbol, timeframe='1h', since=int(timestamp.timestamp()*1000))
    df_new = pd.concat([df, df_since_timestamp]).drop_duplicates(keep='first').reset_index(drop=True)    
    df.index = pd.to_datetime(df.index)
    return df_new
        

def save_ohlcv_data_per_symbol(df, symbol):
    df.to_csv(TICKER_DATA_PATH + f"/{symbol}.csv", index=False)


def load_ohlcv_data_per_symbol(symbol):
    pd.read_csv(TICKER_DATA_PATH + f"/{symbol}.csv")


In this notebook, we load the historical data from Binance for some pairs of interest and save them in a csv file.
We start with data from 2022-01-01.

In [2]:
pairs = pd.read_csv("pairs.csv")
NUM_PAIRS_TO_LOAD = 100
pairs = pairs.iloc[:NUM_PAIRS_TO_LOAD, 0].values
pairs = [pair.replace("USD", "USDT") for pair in pairs]

The below code the history completely from a given start date for a given time-frame.
- To-Do: write another script that updates the history

In [3]:
#pair = 'PEPE/USDT'
INTERVAL = '30m'
START_DATE = pd.to_datetime('2022-01-01')
START_DATE_in_ms = int(START_DATE.timestamp() * 1000)
TIMESTAMP_TODAY = pd.Timestamp.today()
TIMESTAMP_TODAY = pd.Timestamp(year=TIMESTAMP_TODAY.year, month=TIMESTAMP_TODAY.month, day=TIMESTAMP_TODAY.day)

for pair in pairs[13:]:
    df_list = []
    START_DATE_in_ms = int(START_DATE.timestamp() * 1000)
    while True:
        if pair in markets:
            df = fetch_ohlcv_data_per_symbol(symbol=pair, timeframe=INTERVAL, since=START_DATE_in_ms)
        else:
            print(f'error for pair {pair}; Not in Markets!')
            break
        df_list.append(df)
        if df.index[-1].year == TIMESTAMP_TODAY.year and df.index[-1].month == TIMESTAMP_TODAY.month and df.index[-1].day == TIMESTAMP_TODAY.day:
            print(f'done for pair {pair}')
            out_dataframe = pl.concat([pl.from_pandas(df.reset_index()) for df in df_list])
            out_dataframe.write_parquet(f'{TICKER_DATA_PATH}/{pair.replace("/", "")}.parquet')
            del df
            break

        new_start_date = int(df.index[-1].timestamp() * 1000)
        if new_start_date == START_DATE_in_ms:
            print(f'done for pair {pair}, max date reached: {df.index[-1].timestamp()}') 
            out_dataframe = pl.concat([pl.from_pandas(df.reset_index()) for df in df_list])
            out_dataframe.write_parquet(f'{TICKER_DATA_PATH}/{pair.replace("/", "")}.parquet')
            del df
            break
        else:
            START_DATE_in_ms = new_start_date



done for pair NEAR/USDT
done for pair FTM/USDT, max date reached: 1736735400.0
done for pair SHIB/USDT
error for pair USDTC/USDT; Not in Markets!
done for pair LINK/USDT
done for pair BONK/USDT
error for pair POPCAT/USDT; Not in Markets!
done for pair GALA/USDT
done for pair W/USDT
done for pair TAO/USDT
done for pair ALGO/USDT
done for pair FET/USDT
done for pair GBP/USDT, max date reached: 1703817000.0
done for pair TRX/USDT
done for pair SUSHI/USDT
error for pair KAS/USDT; Not in Markets!
done for pair FLOKI/USDT
done for pair LTC/USDT
done for pair ADA/USDT
done for pair ATOM/USDT
done for pair CRV/USDT
error for pair GIGA/USDT; Not in Markets!
done for pair FIL/USDT
done for pair GRT/USDT
done for pair DYDX/USDT
error for pair SAMO/USDT; Not in Markets!
done for pair SYN/USDT
done for pair TIA/USDT
error for pair PONKE/USDT; Not in Markets!
error for pair ATLAS/USDT; Not in Markets!
done for pair MEME/USDT
done for pair DOT/USDT
done for pair UNI/USDT
error for pair ONDO/USDT; Not

In [7]:
pl.read_parquet(f'{TICKER_DATA_PATH}/{pair.replace("/", "")}.parquet')

Date,open,high,low,close,volume,usd_volume
datetime[ns],f64,f64,f64,f64,f64,f64
2022-01-01 00:00:00,1.205,1.2365,1.2041,1.2278,717014.0,880349.7892
2022-01-01 00:30:00,1.2285,1.2517,1.2265,1.251,383167.0,479341.917
2022-01-01 01:00:00,1.251,1.2645,1.2496,1.2592,553802.0,697347.4784
2022-01-01 01:30:00,1.2599,1.2599,1.2429,1.2487,408063.0,509548.2681
2022-01-01 02:00:00,1.2483,1.2543,1.2482,1.2485,163519.0,204153.4715
…,…,…,…,…,…,…
2025-02-03 10:00:00,0.1727,0.1741,0.1714,0.1736,283680.0,49246.848
2025-02-03 10:30:00,0.1738,0.1806,0.1735,0.1801,2.547753e6,458850.3153
2025-02-03 11:00:00,0.1801,0.1801,0.1751,0.1757,955988.0,167967.0916
